This file grabs everything from the LSMS survey that I think an image could possibly recognize and uses those features to predict consumption. This serves as a "gold standard" for any image-based model. Of course, this is no indication of an "upper bound" on CNN performance, but rather offers some means of comparison to a model that was built using only survey data.

I only implement this gold standard for Malawi, but it could be done for the other countries.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
BASE_DIR = '..'

RANDOM_SEED = 7 # for reproducibility
COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

In [3]:
import sys
sys.path.append(BASE_DIR)
from utils import merge_on_lat_lon, assign_groups, run_randomized_cv, run_spatial_cv

In [4]:
def process_malawi():
    np.random.seed(RANDOM_SEED)
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
    consumption_file = 'IHS4 Consumption Aggregate.csv'
    consumption_ph_col = 'rexpagg' # per household
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'HouseholdGeovariables_csv/HouseholdGeovariablesIHS4.csv'
    lat_col = 'lat_modified'
    lon_col = 'lon_modified'

    # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
    ppp = 215.182
    
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_hhf = pd.read_csv(os.path.join(lsms_dir, 'hh_mod_f.csv'))
    df_plot = pd.read_csv(os.path.join(lsms_dir, 'plotgeovariablesihs4.csv'))
    df_com = pd.read_csv(os.path.join(lsms_dir, 'com_cd.csv'))
    df_com2 = pd.read_csv(os.path.join(lsms_dir, 'com_cf1.csv'))
    df_tie = pd.read_csv(os.path.join(lsms_dir, consumption_file))[['case_id', 'ea_id']]

    hhf_input = df_hhf[['case_id', 'hh_f10', 'hh_f08']]
    com_input = df_com[['ea_id', 'com_cd01', 'com_cd16', 'com_cd18a', 'com_cd20a', 'com_cd22a', 'com_cd24a',
                       'com_cd27a', 'com_cd36a', 'com_cd40a', 'com_cd49a', 'com_cd51a', 'com_cd60a', 'com_cd67a',
                       'com_cd69a']]

    com2_input = df_com2[['ea_id', 'com_cf08a']]

    geo_input = df_geo[['case_id', 'dist_admarc', 'dist_agmrkt', 'dist_auction', 'dist_boma', 'dist_borderpost',
                      'dist_popcenter', 'dist_road', 'af_bio_1', 'af_bio_8', 'af_bio_12', 'af_bio_13', 'af_bio_16', 
                       'lat_modified', 'lon_modified']]
    geo_input.rename(columns={'lat_modified': 'cluster_lat', 'lon_modified': 'cluster_lon'}, inplace=True)
    geo_input.dropna(inplace=True)

    plot_input = df_plot[['case_id', 'dist_hh']]
    
    df_cons = pd.read_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed', 'clusters.csv'))
    df_merge = merge_on_lat_lon(df_cons, geo_input)
    df_merge = pd.merge(df_merge, hhf_input, on='case_id', how='left')
    df_merge = pd.merge(df_merge, df_tie, on='case_id', how='left')
    df_merge = pd.merge(df_merge, com_input, on='ea_id', how='left')
    df_merge = pd.merge(df_merge, com2_input, on='ea_id', how='left')
    df_merge = pd.merge(df_merge, plot_input, on='case_id', how='left')
    return df_merge.drop(['case_id', 'ea_id'], axis=1)

In [5]:
df_mw = process_malawi()

/opt/conda/envs/predicting-poverty-replication/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights,dist_admarc,dist_agmrkt,dist_auction,dist_boma,dist_borderpost,...,com_cd27a,com_cd36a,com_cd40a,com_cd49a,com_cd51a,com_cd60a,com_cd67a,com_cd69a,com_cf08a,dist_hh
0,mw,-17.09515,35.217213,1.423239,0.025206,1.0,21.0,145.0,21.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2
1,mw,-17.09515,35.217213,1.423239,0.025206,2.0,20.0,145.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,mw,-17.09515,35.217213,1.423239,0.025206,2.0,20.0,145.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7
3,mw,-17.09515,35.217213,1.423239,0.025206,2.0,20.0,145.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7
4,mw,-17.09515,35.217213,1.423239,0.025206,2.0,20.0,145.0,20.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9


In [7]:
df_mw.shape

(19865, 35)

In [8]:
df_use = pd.get_dummies(df_mw.drop(['country'], axis=1))

In [9]:
clusters = df_use.groupby(['cluster_lat', 'cluster_lon'])

In [10]:
cluster_df = clusters.mean().reset_index()

In [11]:
cluster_df

,cluster_lat,cluster_lon,cons_pc,nightlights,dist_admarc,dist_agmrkt,dist_auction,dist_boma,dist_borderpost,dist_popcenter,...,com_cd27a,com_cd36a,com_cd40a,com_cd49a,com_cd51a,com_cd60a,com_cd67a,com_cd69a,com_cf08a,dist_hh
0,-17.095150,35.217213,1.423239,0.025206,1.500000,20.125000,145.000000,20.125000,4.125000,20.125000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.510000
1,-17.092351,35.114643,1.266204,0.000000,8.105263,25.578947,146.368421,25.578947,10.105263,25.578947,...,4.0,6.0,2.0,NaN,6.0,6.0,45.0,45.0,6.0,0.492308
2,-17.016698,35.079629,1.566870,0.000000,15.761905,23.047619,134.857143,23.047619,21.523810,23.047619,...,0.0,60.0,30.0,NaN,15.0,60.0,60.0,60.0,45.0,0.311765
3,-16.977243,35.205706,1.669245,0.008266,6.970588,11.764706,135.764706,11.764706,13.500000,11.764706,...,1.0,15.0,3.0,NaN,3.0,3.0,15.0,15.0,15.0,2.594118
4,-16.956385,35.168967,1.089891,0.002295,13.000000,13.681818,130.181818,13.681818,20.636364,13.681818,...,500.0,40.0,15.0,NaN,15.0,15.0,40.0,40.0,40.0,0.122222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,-9.591378,33.057450,1.409932,0.000000,7.666667,26.222222,235.277778,26.222222,5.944444,103.666667,...,1.0,3.0,3.0,2.0,NaN,3.0,30.0,30.0,5.0,0.022222
776,-9.550397,33.291558,1.242801,0.000000,10.185185,18.370370,228.740741,18.370370,17.481481,82.481481,...,0.0,21.0,0.0,NaN,7.0,21.0,21.0,21.0,21.0,0.107692
777,-9.519230,33.139193,1.804122,0.003557,5.057143,24.971429,238.400000,24.971429,17.428571,98.828571,...,0.0,41.0,8.0,NaN,8.0,8.0,41.0,NaN,8.0,1.681818
778,-9.507538,33.259649,1.791725,0.000000,4.465116,21.604651,234.209302,21.604651,18.441860,90.000000,...,0.0,5.0,5.0,50.0,NaN,50.0,50.0,50.0,20.0,0.319512


In [12]:
# a few columns have a high percentage of NA
nas = cluster_df.isna().sum() / len(cluster_df)
nas[nas > 0]

com_cd01     0.032051
com_cd16     0.379487
com_cd18a    0.350000
com_cd20a    0.257692
com_cd22a    0.151282
com_cd24a    0.191026
com_cd27a    0.032051
com_cd36a    0.032051
com_cd40a    0.032051
com_cd49a    0.603846
com_cd51a    0.288462
com_cd60a    0.032051
com_cd67a    0.078205
com_cd69a    0.214103
com_cf08a    0.434615
dist_hh      0.042308
dtype: float64

# Modeling

In [13]:
def nan_handler(df):
    nas = df.isna().sum()
    for c in df:
        if nas[c] > 0:
            df[c] = df[c].fillna(df[c].median())
    return df

In [14]:
cleaned_df = nan_handler(cluster_df)

In [15]:
cleaned_df.head()

,cluster_lat,cluster_lon,cons_pc,nightlights,dist_admarc,dist_agmrkt,dist_auction,dist_boma,dist_borderpost,dist_popcenter,...,com_cd27a,com_cd36a,com_cd40a,com_cd49a,com_cd51a,com_cd60a,com_cd67a,com_cd69a,com_cf08a,dist_hh
0,-17.095150,35.217213,1.423239,0.025206,1.500000,20.125000,145.000000,20.125000,4.125000,20.125000,...,2.0,12.0,5.0,10.0,6.0,10.0,20.0,25.0,7.0,1.510000
1,-17.092351,35.114643,1.266204,0.000000,8.105263,25.578947,146.368421,25.578947,10.105263,25.578947,...,4.0,6.0,2.0,10.0,6.0,6.0,45.0,45.0,6.0,0.492308
2,-17.016698,35.079629,1.566870,0.000000,15.761905,23.047619,134.857143,23.047619,21.523810,23.047619,...,0.0,60.0,30.0,10.0,15.0,60.0,60.0,60.0,45.0,0.311765
3,-16.977243,35.205706,1.669245,0.008266,6.970588,11.764706,135.764706,11.764706,13.500000,11.764706,...,1.0,15.0,3.0,10.0,3.0,3.0,15.0,15.0,15.0,2.594118
4,-16.956385,35.168967,1.089891,0.002295,13.000000,13.681818,130.181818,13.681818,20.636364,13.681818,...,500.0,40.0,15.0,10.0,15.0,15.0,40.0,40.0,40.0,0.122222


In [16]:
y = cleaned_df['cons_pc'].values

to_drop = ['cluster_lat', 'cluster_lon', 'cons_pc', 'nightlights']
x = cleaned_df.drop(to_drop, axis=1).values

In [17]:
r2, _ = run_randomized_cv(x, y)

In [18]:
r2

0.08803164394491252

In [19]:
r2, _ = run_randomized_cv(x, np.log(y))

In [20]:
r2

0.46636052584664556

In [21]:
groups, _ = assign_groups(cleaned_df, 5)
r2, _ = run_spatial_cv(x, y, groups)

In [22]:
r2

-1.2040690308579176